In [1]:
# 统计数据
from lib.utils.lmdb_dataset import get_dataset
import lib.model.dp_model as dp
import importlib
from root import get_root_path
import lib.utils.sdata as sd
import os
importlib.reload(dp)
importlib.reload(sd)
code = '600893'
db_path = os.path.sep.join([get_root_path(),'lmdb'])
save_path = os.path.sep.join([get_root_path(),'save'])
source = get_dataset(db_path,code)
sdata = sd.StockData(code, datas=source)
datas = sdata.combine_income(ndays=5,buy=2)


D:\PROJECT_TW\git\finance\lib\utils\sdata.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.datas['Flag'] = list(map(lambda x: 0 if x <= buy else 1, self.datas.INCOME))


In [9]:
# datas.head(10)
datas['DHC'] = (datas.High - datas.Close) * 100 / datas.Close
datas['DAA'] = (datas.Amount - datas.Amount.shift(1)) / datas.Amount
datas.head(10)

d:\project_tw\twedu\venv37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\project_tw\twedu\venv37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Open,High,Close,Low,Volume,Amount,RETURN,INCOME,OUTCOME,Flag,DHC,DCC,DAA
2005-01-05,3.85,3.96,3.94,3.80,783758,304341.5,2.604167,1.522843,-0.253807,0,0.507614,NaN,NaN
2005-01-06,3.88,3.95,3.93,3.85,293276,114618.9,-0.253807,4.071247,1.272265,1,0.508906,-0.254453,-1.655247
2005-01-07,3.95,4.04,3.98,3.93,843481,336057.2,1.272265,4.271357,0.251256,1,1.507538,1.256281,0.658930
2005-01-10,3.98,4.03,4.00,3.94,244178,97395.2,0.502513,3.750000,-0.250000,1,0.750000,0.500000,-2.450449
2005-01-11,4.00,4.05,4.00,3.97,387684,154978.7,0.000000,3.750000,-0.250000,1,1.250000,0.000000,0.371558
2005-01-12,4.00,4.00,3.99,3.90,441909,174203.6,-0.250000,4.010025,0.501253,1,0.250627,-0.250627,0.110359
2005-01-13,3.98,4.18,4.09,3.87,1796552,728164.2,2.506266,1.466993,-4.889976,0,2.200489,2.444988,0.760763
2005-01-14,4.10,4.26,4.15,4.10,3474046,1451579.6,1.466993,-2.650602,-6.265060,0,2.650602,1.445783,0.498364
2005-01-17,4.10,4.10,4.02,3.97,944957,379889.5,-3.132530,1.492537,-3.233831,0,1.990050,-3.233831,-2.821057
2005-01-18,3.99,4.05,4.04,3.96,825610,330563.3,0.497512,0.990099,-3.712871,0,0.247525,0.495050,-0.149219


In [3]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import torch
import torch.nn as nn
np.random.seed(1)
print(datas.head(10))
data,label = sdata.package_data(datas, pdays=15, scale=True)
data = np.array(data)
label = np.array(label)
valid_idx = np.random.randint(10, len(data)-30, int(len(data)*0.1)-30)
valid_idx = np.append(valid_idx,np.array([x for x in range(len(data)-30, len(data))]))
train_idx = np.delete(np.array([x for x in range(len(data))]), valid_idx)
train_data = TensorDataset(torch.tensor(data[train_idx], dtype=torch.float), torch.tensor(label[train_idx], dtype=torch.long))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)

valid_data = TensorDataset(torch.tensor(data[valid_idx], dtype=torch.float), torch.tensor(label[valid_idx], dtype=torch.long))
valid_dataloader = DataLoader(valid_data, shuffle=True, batch_size=16)

            Open  High  Close   Low   Volume     Amount    RETURN    INCOME  \
2005-01-05  3.85  3.96   3.94  3.80   783758   304341.5  2.604167  1.522843   
2005-01-06  3.88  3.95   3.93  3.85   293276   114618.9 -0.253807  4.071247   
2005-01-07  3.95  4.04   3.98  3.93   843481   336057.2  1.272265  4.271357   
2005-01-10  3.98  4.03   4.00  3.94   244178    97395.2  0.502513  3.750000   
2005-01-11  4.00  4.05   4.00  3.97   387684   154978.7  0.000000  3.750000   
2005-01-12  4.00  4.00   3.99  3.90   441909   174203.6 -0.250000  4.010025   
2005-01-13  3.98  4.18   4.09  3.87  1796552   728164.2  2.506266  1.466993   
2005-01-14  4.10  4.26   4.15  4.10  3474046  1451579.6  1.466993 -2.650602   
2005-01-17  4.10  4.10   4.02  3.97   944957   379889.5 -3.132530  1.492537   
2005-01-18  3.99  4.05   4.04  3.96   825610   330563.3  0.497512  0.990099   

             OUTCOME  Flag  
2005-01-05 -0.253807     0  
2005-01-06  1.272265     1  
2005-01-07  0.251256     1  
2005-01-10 -0.

# LSTM 

In [5]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import lib.utils.train_utils as utils

def valid_net(net, dataloader):
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for idx, (inputs, labels) in enumerate(dataloader):
            if use_cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            preds = net(inputs)
            preds = torch.argmax(preds,dim=1)
            
            total += labels.size(0)
            correct += preds.eq(labels).cpu().sum()
        print(idx, len(dataloader), 'Acc: %.3f%% (%d/%d)' % (100. * correct / total, correct, total))
    return correct

learning_rate_decay_start = 50  # 50
learning_rate_decay_every = 5 # 5
learning_rate_decay_rate = 0.9 # 0.9
use_cuda = torch.cuda.is_available()

_epoch = 1000
lr = 0.01
print('data shape --> {}'.format(data.shape))
net = dp.LSTMModel(num_feature=data.shape[2],hidden_size=256,num_class=2)
if use_cuda:
    net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
print('begin .....')
current_correct = 0
for epoch in range(_epoch):
    net.train()
    current_lr = lr
    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate ** frac
        current_lr = lr * decay_factor
        utils.set_lr(optimizer, current_lr)  # set the decayed rate
    else:
        current_lr = lr
    total_loss = 0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()  
        optimizer.zero_grad()
        print('inputs {}'.format(inputs))
        predict = net(inputs)
        loss = criterion(predict, targets)
        total_loss += loss.item()
        loss.backward()
        utils.clip_gradient(optimizer, 0.1)
        optimizer.step()
    print('epcho:{} current lr:{}  loss: {}'.format(epoch, current_lr, total_loss/batch_idx))
    correct = valid_net(net, valid_dataloader)
    if correct > current_correct:
        current_correct = correct
        torch.save(net.state_dict(), '{}/lstm.pkh'.format(save_path))

data shape --> (2990, 15, 7)
begin .....
inputs tensor([[[ 0.9934,  0.9962,  1.0000,  ...,  0.6557,  0.6943,  0.7271],
         [ 1.0000,  1.0000,  0.9924,  ...,  0.6729,  0.7152, -0.4649],
         [ 0.9944,  0.9921,  0.9854,  ...,  0.4439,  0.4670, -0.4259],
         ...,
         [ 0.9456,  0.9336,  0.9299,  ...,  0.9174,  0.9074, -0.2715],
         [ 0.9341,  0.9412,  0.9452,  ...,  1.0000,  1.0000,  1.0000],
         [ 0.9501,  0.9381,  0.9403,  ...,  0.9831,  0.9823, -0.3130]],

        [[ 0.9710,  0.9497,  0.9296,  ...,  0.4218,  0.4040, -0.8371],
         [ 0.9423,  0.9396,  0.9728,  ...,  0.5326,  0.5017,  1.0000],
         [ 0.9713,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  0.6011],
         ...,
         [ 0.9147,  0.8846,  0.9154,  ...,  0.2232,  0.2031,  0.2234],
         [ 0.9217,  0.9064,  0.9147,  ...,  0.3735,  0.3462, -0.0164],
         [ 0.9168,  0.8839,  0.9046,  ...,  0.2208,  0.2001, -0.2377]],

        [[ 0.9914,  1.0000,  1.0000,  ...,  0.8345,  0.8778,  0.1092

         [ 0.8813,  0.8966,  0.9047,  ...,  0.4003,  0.3609,  0.4982]]])
inputs tensor([[[ 0.9848,  0.9826,  0.9675,  ...,  0.5324,  0.5581, -0.4253],
         [ 0.9865,  0.9868,  0.9850,  ...,  0.8990,  0.9509,  1.0000],
         [ 0.9924,  1.0000,  1.0000,  ...,  0.6080,  0.6527,  0.8419],
         ...,
         [ 0.9562,  0.9404,  0.9258,  ...,  1.0000,  1.0000, -1.2642],
         [ 0.9360,  0.9396,  0.9292,  ...,  0.5301,  0.5324,  0.1990],
         [ 0.9470,  0.9338,  0.9175,  ...,  0.6502,  0.6452, -0.6942]],

        [[ 1.0000,  1.0000,  1.0000,  ...,  0.8994,  1.0000, -1.2828],
         [ 0.9701,  0.9732,  0.9630,  ...,  0.8750,  0.9409, -1.8586],
         [ 0.9358,  0.9479,  0.9815,  ...,  0.5059,  0.5436,  0.9650],
         ...,
         [ 0.8925,  0.9003,  0.8565,  ...,  0.8475,  0.8256, -3.6084],
         [ 0.8299,  0.8482,  0.8519,  ...,  1.0000,  0.9362, -0.2713],
         [ 0.8254,  0.8438,  0.8688,  ...,  0.7310,  0.6956,  1.0000]],

        [[ 0.9493,  0.9337,  0.9499,

         [ 0.9042,  0.9093,  0.9101,  ...,  0.2355,  0.2174, -0.5552]]])
inputs tensor([[[ 8.3672e-01,  8.7439e-01,  8.7074e-01,  ...,  5.2746e-01,
           4.5929e-01,  1.7597e-01],
         [ 8.6338e-01,  8.9829e-01,  9.2926e-01,  ...,  5.6205e-01,
           4.9956e-01,  1.0000e+00],
         [ 9.2941e-01,  9.0683e-01,  9.2214e-01,  ...,  5.8268e-01,
           5.3412e-01, -1.1407e-01],
         ...,
         [ 9.9289e-01,  1.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
           1.0000e+00,  2.0339e-01],
         [ 1.0000e+00,  9.7488e-01,  9.8473e-01,  ...,  5.8918e-01,
           5.6871e-01, -2.2715e-01],
         [ 9.7765e-01,  9.3902e-01,  9.6641e-01,  ...,  5.3789e-01,
           5.1202e-01, -2.7680e-01]],

        [[ 9.4912e-01,  9.4278e-01,  9.2432e-01,  ...,  5.6649e-01,
           5.4275e-01, -5.1168e-01],
         [ 9.1887e-01,  9.3018e-01,  9.2705e-01,  ...,  3.1759e-01,
           3.0688e-01,  6.8622e-02],
         [ 9.3159e-01,  9.5402e-01,  9.5240e-01,  ...,  4.7055e-

         [ 0.9899,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]]])
inputs tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  0.2345,  0.2451, -0.0174],
         [ 0.9940,  0.9972,  0.9636,  ...,  0.5691,  0.5795, -1.7717],
         [ 0.9621,  0.9663,  0.9718,  ...,  0.2067,  0.2088,  0.4146],
         ...,
         [ 0.9486,  0.9518,  0.9419,  ...,  0.6150,  0.6059, -0.7803],
         [ 0.9341,  0.9341,  0.9365,  ...,  0.7293,  0.7112, -0.2766],
         [ 0.9309,  0.9384,  0.9330,  ...,  0.5118,  0.5004, -0.1855]],

        [[ 0.6537,  0.6620,  0.7180,  ...,  0.7039,  0.6110,  1.0000],
         [ 0.7244,  0.7281,  0.7897,  ...,  1.0000,  1.0000,  0.9894],
         [ 0.8422,  0.8007,  0.8684,  ...,  0.5464,  0.6012,  0.9881],
         ...,
         [ 0.9246,  0.8711,  0.9201,  ...,  0.1649,  0.1926, -0.0878],
         [ 0.9246,  0.9209,  0.9518,  ...,  0.2110,  0.2605,  0.3417],
         [ 0.9576,  0.9187,  0.9788,  ...,  0.2089,  0.2597,  0.2814]],

        [[ 0.7366,  0.7409,  0.7550,

         [ 1.0000,  0.9902,  0.9872,  ...,  0.3503,  0.3518, -0.2649]]])
inputs tensor([[[ 0.9057,  0.9565,  0.9589,  ...,  1.0000,  1.0000,  1.0000],
         [ 0.9461,  0.9324,  0.9275,  ...,  0.4739,  0.4666, -0.5531],
         [ 0.9218,  0.9140,  0.9302,  ...,  0.3749,  0.3638,  0.0493],
         ...,
         [ 0.9477,  0.9277,  0.9405,  ...,  0.1953,  0.1935, -0.3055],
         [ 0.9391,  0.9361,  0.9324,  ...,  0.3410,  0.3381, -0.1459],
         [ 0.9261,  0.9266,  0.9324,  ...,  0.2522,  0.2498,  0.0000]],

        [[ 0.9585,  0.9530,  0.9520,  ...,  0.2417,  0.2309, -0.2632],
         [ 0.9537,  0.9438,  0.9318,  ...,  0.3202,  0.3005, -0.5320],
         [ 0.9319,  0.9602,  0.9689,  ...,  0.6924,  0.6644,  1.0000],
         ...,
         [ 0.9114,  0.9167,  0.9177,  ...,  0.3625,  0.3325,  0.2903],
         [ 0.9122,  0.9044,  0.8883,  ...,  0.3507,  0.3149, -0.8057],
         [ 0.8884,  0.9000,  0.9109,  ...,  0.2516,  0.2264,  0.6385]],

        [[ 0.8961,  0.8980,  0.9049,

...

         [ 1.0000,  0.9926,  0.9965,  ...,  0.6476,  0.6605, -0.0972]]])
inputs tensor([[[ 0.8699,  0.8549,  0.8660,  ...,  0.3247,  0.2781,  0.5144],
         [ 0.8858,  0.8846,  0.8934,  ...,  0.5160,  0.4574,  1.0000],
         [ 0.9173,  0.8906,  0.8934,  ...,  0.3642,  0.3278,  0.0000],
         ...,
         [ 0.9872,  0.9673,  0.9766,  ...,  0.9827,  0.9533,  0.7738],
         [ 0.9986,  1.0000,  0.9973,  ...,  0.8588,  0.8625,  0.6678],
         [ 1.0000,  0.9997,  1.0000,  ...,  1.0000,  1.0000,  0.0858]],

        [[ 1.0000,  1.0000,  0.9926,  ...,  1.0000,  1.0000, -0.4457],
         [ 0.9901,  0.9646,  0.9720,  ...,  0.7169,  0.6913, -0.8051],
         [ 0.9745,  0.9615,  0.9889,  ...,  0.5276,  0.5110,  0.6741],
         ...,
         [ 0.9437,  0.9209,  0.9510,  ...,  0.2730,  0.2549,  0.2197],
         [ 0.9548,  0.9484,  0.9749,  ...,  0.7195,  0.6881,  0.9747],
         [ 0.9786,  0.9758,  1.0000,  ...,  0.8334,  0.8160,  1.0000]],

        [[ 0.9842,  0.9727,  0.9681,

           6.4308e-01, -2.3249e-01]]])
inputs tensor([[[ 0.9852,  0.9975,  0.9986,  ...,  0.9871,  1.0000,  0.2717],
         [ 1.0000,  1.0000,  0.9994,  ...,  0.6880,  0.6986,  0.0813],
         [ 0.9950,  0.9967,  1.0000,  ...,  0.7654,  0.7740,  0.0541],
         ...,
         [ 0.9675,  0.9719,  0.9750,  ...,  0.5291,  0.5234,  0.6424],
         [ 0.9687,  0.9719,  0.9719,  ...,  0.5117,  0.5070, -0.3052],
         [ 0.9692,  0.9794,  0.9817,  ...,  1.0000,  0.9989,  0.9742]],

        [[ 0.8496,  0.8616,  0.8535,  ...,  0.1608,  0.1421, -0.1096],
         [ 0.8586,  0.8699,  0.8778,  ...,  0.2707,  0.2417,  0.6196],
         [ 0.8891,  0.8755,  0.8757,  ...,  0.2115,  0.1912, -0.0535],
         ...,
         [ 1.0000,  0.9847,  0.9854,  ...,  0.6923,  0.6991, -0.3173],
         [ 0.9948,  0.9893,  0.9759,  ...,  0.4493,  0.4509, -0.2087],
         [ 0.9728,  0.9636,  0.9273,  ...,  0.4972,  0.4848, -1.0837]],

        [[ 0.9714,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],

         [ 0.9257,  0.9326,  0.9437,  ...,  0.3223,  0.3023,  0.2180]]])
inputs tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  0.4222,  0.4363, -0.5161],
         [ 0.9899,  0.9872,  0.9534,  ...,  0.6930,  0.6995, -1.5254],
         [ 0.9466,  0.9758,  0.9825,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [ 0.9004,  0.8974,  0.8936,  ...,  0.4538,  0.4144, -0.7301],
         [ 0.8802,  0.8875,  0.8994,  ...,  0.2962,  0.2713,  0.2134],
         [ 0.8918,  0.8846,  0.8936,  ...,  0.1736,  0.1590, -0.2120]],

        [[ 0.9548,  0.9421,  0.9287,  ...,  0.1936,  0.1832, -0.3592],
         [ 0.9413,  0.9325,  0.9171,  ...,  0.2222,  0.2087, -0.3111],
         [ 0.9281,  0.9160,  0.8947,  ...,  0.1963,  0.1794, -0.6031],
         ...,
         [ 0.9946,  0.9851,  0.9807,  ...,  0.4007,  0.4002,  0.1014],
         [ 0.9959,  0.9884,  0.9807,  ...,  0.3823,  0.3784,  0.0000],
         [ 0.9959,  1.0000,  1.0000,  ...,  0.6995,  0.7053,  0.4882]],

        [[ 0.9882,  0.9801,  0.9899,

         [ 0.7506,  0.7174,  0.7540,  ...,  0.2754,  0.2526,  0.0489]]])


KeyboardInterrupt: 

In [ ]:
data[1]

# GAN